In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
from scipy.stats import mode
from scipy import stats
import feather
import re
import warnings
import random

from IPython.display import clear_output

In [3]:
all_transactions = feather.read_dataframe('all_transactions')

In [4]:
all_transactions = all_transactions[['card_id','purchase_amount','purchase_date','payment_from_end_to_begging']]

In [5]:
all_transactions_sorted = all_transactions.sort_values(by='purchase_date')

In [6]:
def distance_between_purchases_max(x):
    return (x.diff().dropna().dt.seconds // 60).max()

def distance_between_purchases_min(x):
    return (x.diff().dropna().dt.seconds // 60).min()

def distance_between_purchases_mean(x):
    return (x.diff().dropna().dt.seconds // 60).mean()

def distance_between_purchases_median(x):
    return (x.diff().dropna().dt.seconds // 60).median()

def distance_between_purchases_ptp(x):
    return (x.diff().dropna().dt.seconds // 60).ptp()

def distance_between_purchases_skew(x):
    return (x.diff().dropna().dt.seconds // 60).skew()

def distance_between_purchases_kurt(x):
    return (x.diff().dropna().dt.seconds // 60).kurt()

In [7]:
by_all_purchases = all_transactions_sorted.groupby(['card_id']).agg({'purchase_date':[distance_between_purchases_max,distance_between_purchases_min,distance_between_purchases_mean,distance_between_purchases_median,distance_between_purchases_ptp,distance_between_purchases_kurt,distance_between_purchases_skew]})

In [8]:
only_big_purch = all_transactions_sorted[all_transactions_sorted.payment_from_end_to_begging == 0].groupby(['card_id']).agg({'purchase_date':[distance_between_purchases_max,distance_between_purchases_min,distance_between_purchases_mean,distance_between_purchases_median,distance_between_purchases_ptp,distance_between_purchases_kurt,distance_between_purchases_skew]})

In [9]:
by_all_purchases = pd.merge(by_all_purchases,only_big_purch,how='left',on='card_id')

In [10]:
by_all_purchases.fillna(0,inplace=True)

In [11]:
by_all_purchases.columns = [f'all_months_{x[1]}' if x[0][-1]=='x' else f'last_month_{x[1]}' for x in by_all_purchases]

In [12]:
by_all_purchases.reset_index(inplace=True)

In [13]:
feather.write_dataframe(by_all_purchases,'all_purchases_diff')